In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm 

ModuleNotFoundError: No module named 'calculate_delta_vega'

In [2]:
days = 5

underlying_prices = list([377.85, 369.14, 372.52, 368.8, 374.23])
call_option_prices = list([18.9455, 14.49, 11.65, 13.30, 15.45])
option_deltas = list([0.5072, 0.3418, 0.4026, 0.3280, 0.4146])

S = 374.23      # Current stock price
K = 380      # Option strike price
T = 1        # Time to expiration in years
r = 0.05     # Risk-free interest rate
sigma = 0.2  # Volatility

option_type = 'call'  # or 'put'

delta_value = delta(S, K, T, r, sigma, option_type)

data = pd.DataFrame({
    'Day': np.arange(1, days + 1),
    'Underlying Price': underlying_prices,
    'Call Option Price': call_option_prices,
    'Option Delta': option_deltas
})

NameError: name 'delta' is not defined

In [18]:
def delta_hedge(price_of_underlying, price_of_call_option, delta_of_option):
    """
    Calculate the amount of underlying asset needed to delta hedge a call option position.

    :param price_of_underlying: Current price of the underlying asset.
    :param price_of_call_option: Current price of the call option.
    :param delta_of_option: Delta of the call option.
    :return: Amount of underlying asset to hedge the option position.
    """
    hedge_amount = -delta_of_option * price_of_call_option / price_of_underlying
    return hedge_amount

data['Hedge Amount'] = data.apply(lambda row: delta_hedge(row['Underlying Price'], 
                                                         row['Call Option Price'], 
                                                         row['Option Delta']), axis=1)

data.head()



,Day,Underlying Price,Call Option Price,Option Delta,Hedge Amount
0,1,377.85,18.9455,0.5072,-0.025431
1,2,369.14,14.4900,0.3418,-0.013417
2,3,372.52,11.6500,0.4026,-0.012591
3,4,368.80,13.3000,0.3280,-0.011829
4,5,374.23,15.4500,0.4146,-0.017117


In [20]:
def calculate_Ai(Ci_plus_1, Ci, Delta_i, Si_plus_1, Si):
    """
    Calculate the difference Ai between the change in value of OP and RE.

    :param Ci_plus_1: Price of the call option at time i+1.
    :param Ci: Price of the call option at time i.
    :param Delta_i: Delta of the option at time i.
    :param Si_plus_1: Price of the underlying asset at time i+1.
    :param Si: Price of the underlying asset at time i.
    :return: Difference Ai.
    """
    return (Ci_plus_1 - Ci) - Delta_i * (Si_plus_1 - Si)

data['A_i'] = calculate_Ai(data['Call Option Price'].shift(-1), 
                          data['Call Option Price'], 
                          data['Option Delta'], 
                          data['Underlying Price'].shift(-1), 
                          data['Underlying Price'])

error = (data['A_i'] ** 2).mean()

print("Total Mean Squared Error:", error)

data

Total Mean Squared Error: 6.501923168695977


,Day,Underlying Price,Call Option Price,Option Delta,Hedge Amount,A_i
0,1,377.85,18.9455,0.5072,-0.025431,-0.037788
1,2,369.14,14.4900,0.3418,-0.013417,-3.995284
2,3,372.52,11.6500,0.4026,-0.012591,3.147672
3,4,368.80,13.3000,0.3280,-0.011829,0.368960
4,5,374.23,15.4500,0.4146,-0.017117,NaN
